<a href="https://colab.research.google.com/github/JeongJeong-code/P6_market_place/blob/main/text_classificationV2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [97]:
import sys
import numpy as np
import matplotlib.pyplot as plt
from enum import Enum
from google.colab import files
import pandas as pd
import seaborn as sns
import os.path
import missingno as mi
import re
import sklearn as sk
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics.cluster import adjusted_rand_score
import statsmodels.api as sm
from statsmodels.formula.api import ols
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.manifold import TSNE
from sklearn.metrics.cluster import homogeneity_score
from sklearn.decomposition import KernelPCA
from sklearn.cluster import (AgglomerativeClustering,
                             KMeans, DBSCAN, SpectralClustering)
from sklearn.metrics import davies_bouldin_score, silhouette_score
! {sys.executable} -m pip install ipynb

import cv2
from tqdm import tqdm

In [98]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [99]:
url = 'https://raw.githubusercontent.com/JeongJeong-code/P6_market_place/main/df1_test.csv'
df1_test= pd.read_csv(url)


In [100]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
import nltk.stem
import string

In [101]:
from nltk.tokenize import RegexpTokenizer

tokenizer = RegexpTokenizer(r'\w+')
tokenize_desc = pd.Series(df1_test.description.str.lower().apply(tokenizer.tokenize),name ='tokenize_desc')

In [102]:
all_words = [word for tokens in tokenize_desc for word in tokens]
from collections import Counter
count_all_words = Counter(all_words)
#count_all_words.most_common(100)

In [103]:
stopword_list = nltk_stop_words = nltk.corpus.stopwords.words('english') + list(count_all_words.most_common(100))


In [104]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
y = pd.Series(le.fit_transform(df1_test.categories),name='class_labels')




In [105]:
list_labels = y
list_tokens = tokenize_desc

In [106]:
list_tokens

0      [key, features, of, elegance, polyester, multi...
1      [specifications, of, sathiyas, cotton, bath, t...
2      [key, features, of, eurospa, cotton, terry, fa...
3      [key, features, of, santosh, royal, fashion, c...
4      [key, features, of, jaipur, print, cotton, flo...
                             ...                        
923    [oren, empower, extra, large, self, adhesive, ...
924    [wallmantra, large, vinyl, sticker, sticker, p...
925    [buy, uberlyfe, extra, large, pigmented, polyv...
926    [buy, wallmantra, medium, vinyl, sticker, stic...
927    [buy, uberlyfe, large, vinyl, sticker, for, rs...
Name: tokenize_desc, Length: 928, dtype: object

In [107]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(df1_test.description,df.class_labels, test_size=0.1, random_state=40)

In [108]:
X_train[:10]

787    Woven Terry Cotton Bath Towel (Bath Towel, Mul...
104    Specifications of Toffyhouse Baby Boy's Sleeps...
600    Buy Welhouse Cotton Hand & Face Towel Set at R...
454    Sonata 77036SM02J Watch - Buy Sonata 77036SM02...
31     Lenco Bdblue Tango Analog Watch  - For Men, Bo...
262    Specifications of Yves Bertelin YBSCR557 Analo...
253    Ndura Kadhai 1.7 L\r\n                        ...
92     Artisan Creation Checkered Single Quilts & Com...
370    Buy Zyxel VMG1312-B10A VDSL2 Wireless N VDSL2 ...
204    Agromech YIBOO Rolling Pizza Cutter (Stainless...
Name: description, dtype: object

In [109]:
from sklearn.feature_extraction.text import CountVectorizer

count_vectorizer = CountVectorizer(analyzer='word', token_pattern=r'\w+',stop_words=stopword_list)

bow = dict()
bow["train"] = (count_vectorizer.fit_transform(X_train), y_train)
bow["test"]  = (count_vectorizer.transform(X_test), y_test)
print(bow["train"][0].shape)
print(bow["test"][0].shape)

(835, 4382)
(93, 4382)


In [110]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(analyzer='word', token_pattern=r'\w+',stop_words=stopword_list)

tfidf = dict()
tfidf["train"] = (tfidf_vectorizer.fit_transform(X_train), y_train)
tfidf["test"]  = (tfidf_vectorizer.transform(X_test), y_test)

In [111]:
from sklearn.naive_bayes import MultinomialNB
nb_classifier = MultinomialNB()

In [115]:
nb_classifier.fit(*bow["train"])

MultinomialNB()

In [118]:
from sklearn.model_selection import cross_val_score
cross_val_score(nb_classifier, *bow["train"], cv=5, scoring='accuracy')

array([0.75449102, 0.77245509, 0.78443114, 0.79041916, 0.77245509])